In [2]:
import numpy as np
from collections import defaultdict

In [ ]:
L = input("Please enter the number of layers: ")
h = input("Please enter the number of hashes per layer: ")
t = input("Please enter the number of similar videos: ")
queryVideo = input("Enter the query video id: ")

### **TASK 3**

**Part A:**
Implement a Locality Sensitive Hashing (LSH) tool for Euclidean distance

INPUT:
* L = number of layers
* h = number of hashes per layer
* set of vectors (all target and non-target videos for a visual model (256 dimensions))

OUPUT:
* In-memory index structure containing the given set of vectors

***

**Part B:**
Implement a similar video search tool using the index
structure storing all target and non-target videos for a visual model
of your choice.

INPUT:
* v = videoID
* t = number of similar videos

OUPUT:
* list of videos in order of decreasing order of similarity
* thumbnails of the most similar t videos
* number of unique and overall video candidates considered during the process

In [ ]:
# TODO: change random to set of vectors (all target and non-target videos for a visual model (256 dimensions))

# Global variables for LSH
layers = []
buckets = []

# Initialize LSH structure with L layers and h hash functions per layer.
def initialize_lsh(L, h):

    global layers, buckets
    layers = [[np.random.randn(256) for _ in range(h)] for _ in range(L)]
    buckets = [defaultdict(list) for _ in range(L)]

# Hash a vector using a set of hyperplanes.
def hash_vector(vector, hyperplanes):

    return tuple(int(np.dot(vector, plane) >= 0) for plane in hyperplanes)

# Add a vector to the LSH structure.
def add_vector_to_lsh(vector, vector_id):

    for i, hyperplanes in enumerate(layers):
        hash_value = hash_vector(vector, hyperplanes)
        buckets[i][hash_value].append(vector_id)

# Build the LSH index for a set of vectors.
def build_lsh_index(vectors):

    for vector_id, vector in enumerate(vectors):
        add_vector_to_lsh(vector, vector_id)

# Query the LSH structure for similar vectors.
def query_lsh(vector):

    candidates = set()
    for i, hyperplanes in enumerate(layers):
        hash_value = hash_vector(vector, hyperplanes)
        candidates.update(buckets[i][hash_value])
    return list(candidates)


In [7]:
# TODO: change distance with similarity score
# TODO: add thumbnails

# Search the LSH index for the most similar videos to a query video ID.
def video_search_lsh(query_video_id, vectors, t):

    # Retrieve the query vector
    query_vector = vectors[query_video_id]
    
    # Get candidate vector IDs using LSH
    candidate_ids = query_lsh(query_vector)
    
    # Compute Euclidean distances between the query vector and the candidates
    similarities = []
    for vector_id in candidate_ids:
        distance = np.linalg.norm(query_vector - vectors[vector_id])
        similarities.append((vector_id, distance))
    
    # Sort by distance (ascending)
    sorted_candidates = sorted(similarities, key=lambda x: x[1])
    
    # Get the top t most similar videos
    top_t = sorted_candidates[:t]

    # Statistics
    unique_candidates = len(set(candidate_ids))  # Unique candidates retrieved
    overall_candidates = len(candidate_ids)  # Total candidates retrieved

    return top_t, unique_candidates, overall_candidates


In [6]:
L = 5  # Number of layers
h = 4  # Number of hash functions per layer
num_vectors = 100  # Number of vectors to index
t = 5  # Number of most similar videos to retrieve

# Generate random vectors
np.random.seed(42)
vectors = np.random.randn(num_vectors, 256)

# Initialize LSH
initialize_lsh(L, h, 256)

# Build the index
build_lsh_index(vectors)

# Query for the first vector
query_video_id = 0  # Video ID to query
results, unique_count, overall_count = video_search_lsh(query_video_id, vectors, t)

print(f"Top {t} similar videos to video ID {query_video_id}:")
for rank, (video_id, distance) in enumerate(results, start=1):
    print(f"{rank}: Video ID {video_id}, Distance: {distance:.4f}")
print(f"Unique candidates: {unique_count}, Overall candidates: {overall_count}")

Top 5 similar videos to video ID 0:
1: Video ID 0, Distance: 0.0000
2: Video ID 88, Distance: 20.7514
3: Video ID 40, Distance: 20.8395
4: Video ID 94, Distance: 21.0309
5: Video ID 98, Distance: 21.1167
Unique candidates: 23, Overall candidates: 23
